# <span style="color: green; font-size: 40px; font-weight: bold;"> Projeto (Regressão) </span>

<br> <br>

# Prevendo a Cotação de Criptomoedas em Tempo Real com PySpark e Machine Learning

<br>

### Contexto

Neste mini-projeto, vamos explorar um importante contexto de negócio na área de finanças: a **previsão da cotação de criptomoedas**. O projeto será desenvolvido desde a concepção do problema de negócio até a entrega de um modelo preditivo, utilizando ferramentas comuns de análise de dados no dia a dia de um Cientista de Dados. Apesar do foco ser em ferramentas de análise de dados, o projeto não serve como aconselhamento financeiro.

<br>

### Objetivo

O objetivo deste mini-projeto é **construir um modelo de Machine Learning capaz de prever a cotação de criptomoedas**. Usaremos dados históricos do Bitcoin para treinar o modelo. O Bitcoin, lançado em 2009 pelo anônimo Satoshi Nakamoto, é a criptomoeda mais antiga e conhecida. Servindo como meio descentralizado de troca digital, as transações de Bitcoin são verificadas e registradas em um livro público distribuído chamado Blockchain. O modelo deve ser capaz de prever a cotação do Bitcoin em tempo real a partir de novos dados de entrada. Este projeto pode ser estendido para outras criptomoedas ou instrumentos financeiros com dados de cotação disponíveis.

<br>

### Pergunta de Negócio Principal

> "Como podemos prever a cotação futura do Bitcoin usando dados históricos?"

<br>

### Entregável

O entregável deste mini-projeto será um **modelo de Machine Learning treinado para prever a cotação do Bitcoin**. O modelo será desenvolvido utilizando dados históricos de cotação do Bitcoin e será capaz de fazer previsões em tempo real com base em novos dados de entrada. O processo incluirá a concepção do problema de negócio, preparação dos dados, desenvolvimento do modelo, e a entrega do modelo preditivo.

<br>

### Sobre o Conjunto de Dados

Os dados utilizados neste mini-projeto abrangem o período de 2011 a 2021. O arquivo CSV contém registros OHLC (Open, High, Low, Close) da cotação do Bitcoin, Volume em BTC e Volume na moeda (dólar).

A última coluna indica o preço ponderado do Bitcoin. Os carimbos de data/hora (timestamp) estão em hora Unix. Timestamps sem atividade têm seus campos de dados preenchidos com NaNs. Se estiver faltando um carimbo de data/hora ou houver saltos, isso pode ser devido à inatividade da Exchange, inexistência da Exchange, ou outros erros técnicos na coleta dos dados.

Optamos por não usar dados do ano de 2022 devido à sua natureza atípica, mas você pode incluir esses dados e treinar novamente o modelo se desejar.

<br>

Para este projeto, utilizaremos o conjunto de dados "Bitcoin Historical Data", que contém quase 5 milhões de linhas e informações detalhadas sobre as transações de Bitcoin ao longo do tempo. O conjunto de dados inclui dados sobre preços de abertura, fechamento, máximo e mínimo, volume de Bitcoins negociados, volume em moeda fiduciária e preço ponderado. Além disso, uma coluna adicional dateTime foi criada para converter os timestamps Unix em um formato de data e hora legível.

<br>

<table border="2">
  <tr>
    <th style="text-align: center; font-size: 16px;">Nome da Coluna</th>
    <th style="text-align: center; font-size: 16px;">Tipo de Dado</th>
    <th style="text-align: center; font-size: 16px;">Descrição</th>
  </tr>
  <tr>
    <td>Timestamp</td>
    <td>integer</td>
    <td>Representa o Unix timestamp, que é o número de segundos desde 1 de janeiro de 1970 (UTC).</td>
  </tr>
  <tr>
    <td>Open</td>
    <td>double</td>
    <td>Preço de abertura do Bitcoin no início do período de tempo.</td>
  </tr>
  <tr>
    <td>High</td>
    <td>double</td>
    <td>Preço mais alto do Bitcoin durante o período de tempo.</td>
  </tr>
  <tr>
    <td>Low</td>
    <td>double</td>
    <td>Preço mais baixo do Bitcoin durante o período de tempo.</td>
  </tr>
  <tr>
    <td>Close</td>
    <td>double</td>
    <td>Preço de fechamento do Bitcoin no final do período de tempo.</td>
  </tr>
  <tr>
    <td>Volume_(BTC)</td>
    <td>double</td>
    <td>Volume total de Bitcoins negociados durante o período de tempo.</td>
  </tr>
  <tr>
    <td>Volume_(Currency)</td>
    <td>double</td>
    <td>Volume total em moeda fiduciária (por exemplo, USD) das transações de Bitcoin durante o período.</td>
  </tr>
  <tr>
    <td>Weighted_Price</td>
    <td>double</td>
    <td>Preço ponderado do Bitcoin, calculado com base nos preços e volumes das transações durante o período.</td>
  </tr>
  <tr>
    <td>dateTime</td>
    <td>string</td>
    <td>Irá representar a data e hora no formato legível, será criada a partir do Unix timestamp. <b>(Nova Coluna)</b> </td>
  </tr>
</table>


<br> <br> <br>

# Importando Pacotes

In [5]:
### Imports

# Importa o findspark e inicializa
import findspark
findspark.init()

## Bibliotecas de Manipulação e Análise de Dados

import pandas as pd                      # Biblioteca para manipulação e análise de dados tabulares.
import numpy as np                       # Biblioteca para cálculos numéricos e manipulação de arrays.


## Bibliotecas de Visualização de Dados

import seaborn as sns                    # Biblioteca para visualização de dados estatísticos.
from matplotlib import pyplot as plt     # Biblioteca para criação de gráficos e visualizações.


## Bibliotecas Principais do PySpark

import pyspark                           # Biblioteca para processamento de dados em grande escala usando clusters.
from pyspark import SparkConf            # Configuração e criação do contexto do Spark.
from pyspark import SparkContext         # Configuração e criação do contexto do Spark.   
from pyspark.sql import SparkSession     # Criação e manipulação de sessões e contextos SQL no Spark.
from pyspark.sql import SQLContext       # Criação e manipulação de sessões e contextos SQL no Spark.
from pyspark.sql.types import *          # Tipos de dados usados na criação de schemas de DataFrames no Spark.
from pyspark.sql.functions import *      # Funções SQL usadas para manipulação e transformação de dados no Spark.

## Bibliotecas de Machine Learning no PySpark

from pyspark.ml.linalg import Vectors         # Estruturas de dados para manipulação de vetores na MLlib do Spark.
from pyspark.ml.feature import StringIndexer  # Transformação de variáveis categóricas em numéricas.
from pyspark.ml.regression import LinearRegression      # Algoritmo de regressão linear na MLlib do Spark.
from pyspark.mllib.evaluation import RegressionMetrics  # Métricas de avaliação para modelos de regressão.
from pyspark.ml.stat import Correlation                 # Cálculo de correlações entre colunas de DataFrames.
from pyspark.ml.feature import MinMaxScaler             # Normalização dos dados para um intervalo específico.
from pyspark.ml.feature import VectorAssembler   # Combinação de múltiplas colunas em uma única coluna de vetores.
from pyspark.ml import Pipeline          # Construção de pipelines de ML que consistem em uma sequência de etapas.
from pyspark.ml.tuning import ParamGridBuilder    # Ferramentas para construção de grids de parâmetros.
from pyspark.ml.tuning import CrossValidator      # Ferramentas para construção de validação cruzada.
from pyspark.ml.tuning import CrossValidatorModel # Ferramentas para construção de validação cruzada.
from pyspark.ml.feature import StandardScaler     # Normalização de dados para ter média zero e variância unitária.
from pyspark.ml.evaluation import RegressionEvaluator  # Avaliação de modelos de regressão utilizando 
                                                       # métricas como RMSE e R2.

In [2]:
## Formatação das saídas

# Configura o Pandas para mostrar até 200 colunas ao exibir um DataFrame
pd.set_option('display.max_columns', 200)

# Configura o Pandas para mostrar até 400 caracteres por coluna ao exibir um DataFrame
pd.set_option('display.max_colwidth', 400)

# Configuramos matplotlib_axes_logger para exibir apenas mensagens de erro 
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

<br>

## Por que vamos usar o <i>PySpark</i> ao invés de utilizarmos somente <i>Linguagem Python</i>?

> Antes de responder, podemos afirmar que SIM, este projeto poderia ser feito usando somente <i>Linguagem Python</i>.

#### Então por que usar o PyStark?

Como iremos ver a seguir, nosso conjunto de dados possui um tamanho de **317MB** e **quase 5 milhões de linhas**. Será que conseguiríamos processar esse volume de dados tão alto com Linguagem Python? Provavelmente não!

<br>

Portanto usaremos o **PySpark** pois o ele nos permite trabalhar em um ambiente distribuído. 

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

sys       : 3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
pyspark   : 3.5.1
findspark : 2.0.1
numpy     : 1.22.4
decimal   : 1.70
matplotlib: 3.4.3
seaborn   : 0.11.2
pandas    : 1.3.5



<br>

# Preparando o Ambiente Spark

In [6]:
# Definindo semente aleatória (seed) para reprodutibilidade do notebook
rnd_seed = 23
np.random.seed = rnd_seed
np.random.set_state = rnd_seed

# Criando o Spark Context
sc = SparkContext(appName = "Mini-Projeto3")

24/07/10 18:35:36 WARN Utils: Your hostname, eduardo-Inspiron-15-3520 resolves to a loopback address: 127.0.1.1; using 192.168.0.13 instead (on interface wlp0s20f3)
24/07/10 18:35:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/10 18:35:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
# Criando a sessão Spark
spark_session = SparkSession.Builder().getOrCreate()

# Visualiza o objeto spark_session
spark_session